* Παναγιώτης Ζευγολατάκος 03117804
* Χατζηθεοδώρου Ιάσων 03117089
* Ανδρέας Βεζάκης 03117186

# Εφαρμογή 1

In [ ]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from sklearn.cluster import KMeans
from tabulate import tabulate
from gensim.models import Word2Vec
import gensim.downloader
from gensim.parsing.preprocessing import remove_stopwords, strip_non_alphanum, strip_short
import somoclu
import matplotlib
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer, LancasterStemmer
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
%matplotlib inline

import joblib
import random
import string
from copy import deepcopy
import re
import multiprocessing

## Διαβασμα αρχείων

In [ ]:
dataset_url = "https://drive.google.com/uc?export=download&id=1-tN5rC1QCgwqL67cvIXdMbtd-Xf9Xqow"
df_data_1 = pd.read_csv(dataset_url, sep='\t',  header=None, quoting=3)

# Στο επόμενη γραμή βάλτε τον αριθμό της ομάδας στο εργαστήριο των νευρωνικών
team_seed_number = 13

movie_seeds_url = "https://drive.google.com/uc?export=download&id=1-oeMf7BFmiH70XObgC9WOTo-a9cDJZwP"
df_data_2 = pd.read_csv(movie_seeds_url, header=None)

# επιλέγεται 
my_index = df_data_2.iloc[team_seed_number,:].values

titles = df_data_1.iloc[:, [2]].values[my_index] # movie titles (string)
categories = df_data_1.iloc[:, [3]].values[my_index] # movie categories (string)
bins = df_data_1.iloc[:, [4]]
catbins = bins[4].str.split(',', expand=True).values.astype(float)[my_index] # movie categories in binary form (1 feature per category)
summaries =  df_data_1.iloc[:, [5]].values[my_index] # movie summaries (string)
corpus = summaries[:,0].tolist() # list form of summaries
corpus_df = pd.DataFrame(corpus) # dataframe version of corpus

In [ ]:
ID = 999
print(titles[ID])
print(categories[ID])
print(catbins[ID])
print(corpus[ID])

['The Photographer']
['"Comedy",  "Indie"']
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 

## Προεπεξεργασία

Αρχικά, στην προεπεξεργασία μας του corpus, είναι ο καθαρισμός των περιγραφών των ταινιών. Συγκεκριμένα υλοποιήσαμε:
- Μετατροπή όλων των χαρακτήρων σε πεζά
- Αφαίρεση stopwords, σημείων στίξης και ειδικών χαρακτήρων
- Αφαίρεση σύντομων συμβολοσειρών

Ακόμη, σε ξεχωριστό cell, δοκιμάσαμε να εφαρμόσουμε stemming (μέθοδος κανονικοποίησης), για περαιτέρω δοκιμές.

In [ ]:
#nltk.download('punkt')
def stemSentence(sentence):
    porter = PorterStemmer()
    lancaster=LancasterStemmer()
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        #stem_sentence.append(lancaster.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [ ]:
corpus_preprocessed = deepcopy(corpus)

# Lowercase
corpus_preprocessed = [d.lower() for d in corpus_preprocessed]

# Punctuation
corpus_preprocessed = [re.sub('[%s]' % re.escape(string.punctuation), '' , d) for d in corpus_preprocessed]

# Digits
corpus_preprocessed = [strip_non_alphanum(d) for d in corpus_preprocessed]

# Short words
corpus_preprocessed = [strip_short(d) for d in corpus_preprocessed]

# Stopwords
corpus_preprocessed = [remove_stopwords(d) for d in corpus_preprocessed]

#corpus_preprocessed = [stemSentence(d) for d in corpus_preprocessed]

corpus_preprocessed = np.array(corpus_preprocessed)

In [ ]:
corpus[0]

'During the Prohibition era, a young widow, Claire, gets involved in liquor smuggling and romance with two men, Walker and Kibby, off the San Diego coast. Organized crime controls bootlegging back east and wants to do the same here, so a hit man named McTeague is sent to deal with these amateur crooks, as is the Coast Guard, leading to various battles at sea.'

In [ ]:
corpus_preprocessed[0]

'prohibition era young widow claire gets involved liquor smuggling romance men walker kibby san diego coast organized crime controls bootlegging east wants hit man named mcteague sent deal amateur crooks coast guard leading battles sea'

In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
corpus_tf_idf_plain = vectorizer.transform(corpus)

In [ ]:
print(corpus_tf_idf_plain.shape)

(5000, 49146)


## Υλοποίηση του συστήματος συστάσεων

In [ ]:
def content_recommender(target_movie,max_recommendations,corpus_type):
  #cos_sim = []
  #for i in range(corpus_type.shape[0]):   
  #  cos_sim.append(cosine_similarity(corpus_type[target_movie],corpus_type[i]))

  corpus_arr = corpus_type.toarray()
  cos_sim = cosine_similarity(corpus_arr, [corpus_arr[target_movie]])

  sorted_arr = sorted(range(len(cos_sim)), key = lambda k : cos_sim[k], reverse=True)

  recommendations = []
  for i in range(max_recommendations):
    recommendations.append((sorted_arr[i],titles[sorted_arr[i]][0],corpus[sorted_arr[i]][:45]+" ...",categories[sorted_arr[i]][0][:]))
  
  print(tabulate(recommendations,headers=["Movie_ID","Title", "Corpus","Categories"]))

In [ ]:
content_recommender(17,10,corpus_tf_idf_plain)

  Movie_ID  Title             Corpus                                             Categories
----------  ----------------  -------------------------------------------------  -----------------------------------------------------------------------------------------------------
        17  Dogora            As an amorphous alien lifeform annihilated a  ...  "Science Fiction",  "Japanese Movies",  "Creature Film",  "World cinema"
      2416  Rodan             In the small mining village of Kitamatsu, on  ...  "Monster movie",  "Science Fiction",  "Horror",  "World cinema",  "Creature Film",  "Japanese Movies"
      1653  The Animatrix     The plot-summaries of the shorts are listed b ...  "Science Fiction",  "Japanese Movies",  "Adventure",  "Animation",  "Anime",  "Drama",  "Action"
       423  Executive Action  The movie starts with a voice over stating th ...  "Crime Fiction",  "Action/Adventure",  "Drama",  "Political thriller",  "Action",  "Political drama"
      4712  Days of Darkness

## Βελτιστοποίηση του TfidfVectorizer

Για την εύρεση των καλύτερων τιμών των παραμέτρων της TfidfVectorizer, θα πρέπει για κάθε τιμή να συγκρίνουμε τις ταινίες που προτείνει το σύστημα συστάσεων, και να καταλήξουμε σε αυτές που θεωρούμε πιο σχετικές μεταξύ τους. Δοκιμάζοντας αρκετές διαφορετικές τιμές, και διαφορετικές παραμέτρους, παρατηρούμε ότι στις περισσότερες περιπτώσεις, οι ταινίες που προτείνονται είναι αρκετά παρόμοιες με την αρχική ταινία-στόχο. Όμως υπάρχουν και ταινίες στόχοι των οποίων η κατηγοροποίηση δεν είναι τόσο εύκολη και επομένως οι συστάσεις έχουν αρκετά μεγάλη απόκλιση. <br>


## Βαθιά μάθηση: δημιουργία corpora με χρήση word embeddings

In [ ]:
print(list(gensim.downloader.info()['models'].keys())) 

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
#twitter50 = gensim.downloader.load("glove-twitter-50")
#joblib.dump(twitter50, 'model_twitter-50')

In [ ]:
twitter50 = joblib.load('model_twitter-50')
twitter50.vectors.shape

(1193514, 50)

In [ ]:
def build_tfw2v(corpus, vectors, embeddings_size):
    vectorizer = TfidfVectorizer()
    corpus_tf_idf = vectorizer.fit_transform(corpus).toarray()
    vocab = vectorizer.vocabulary_

    # Split
    corpus_split = [d.split() for d in corpus]

    #w2v = np.array((corpus.shape[0], embeddings_size))
    w2v = []
    missed_words = []
    for idx, description in enumerate(corpus_split):
        nom = 0
        denom = 0
        for word in description:
            # Skip words that are not in the embeddings
            if word in vectors:
                word_index = vocab[word]
                tf_idf = corpus_tf_idf[idx][word_index]
                nom += tf_idf * vectors[word]
                denom += tf_idf
            else: 
                missed_words.append(word)
        w2v.append(nom * 1/denom)
    return sparse.csr_matrix(np.array(w2v)), missed_words

In [ ]:
w2v, missed = build_tfw2v(corpus_preprocessed, twitter50, 50)
w2v.shape

(5000, 50)

In [ ]:
len(missed)

36586

## Ανάλυση Αποτελεσμάτων

### Σύστημα συστάσεων βασισμένο μόνο στο TF IDF

Παρακάτω εκτελείται ο αλγόριθμος για τις ταινίες με ID = 5 και 17 και επιστρέφει ικανοποιητικά αποτελέσματα. <br>
Για την ταινία "Alms for a Blind Horse", είναι πολύ εύκολο να διαπιστώσουμε ότι οι προτείνομενες ταινίες είναι αρκετά παρόμοιες, αφού σχεδόν όλες είναι "Drama". <br>



In [ ]:
vectorizer = TfidfVectorizer(lowercase=True,max_df=100,min_df=10,max_features=10000)
vectorizer.fit(corpus)
corpus_tf_idf_plain = vectorizer.transform(corpus)
print(corpus_tf_idf_plain.shape)
content_recommender(5,10,corpus_tf_idf_plain)

(5000, 7239)
  Movie_ID  Title                         Corpus                                             Categories
----------  ----------------------------  -------------------------------------------------  ---------------------------------------------------------------------------------
         5  Alms for a Blind Horse        The film is set in a village in the outskirts ...  "Drama"
      4599  Mannu                         Mannu is about the struggle between Damu, a t ...  "Drama"
       375  Silver Stallion               Soldiers with the U.N. forces that entered Ko ...  "Western",  "Drama",  "Adventure"
      1191  Boni                          DD  Chinna  are two close friends who are bro ...  "Romance Film",  "Action",  "Adventure",  "Comedy"
      1153  Kannezhuthi Pottum Thottu     A young woman named Bhadra  wants to take rev ...  "Drama"
      1188  Summer 2007                   Rahul , Priyanka , Qateel , Bagani  and Visha ...  "Thriller",  "Crime Fiction",  "World cin

Για την ταινία "Dogora" παρατητούμε ότι η κατηγορία "Science Fiction" χαρακτηρίζει μέχρι και την 10η σύσταση, ενώ οι "Horror","Thriller","Creature Film","Drama" που χαρακτηρίζουν καλά τη ταινία-στόχο μας, εμφανίζονται πολύ συχνά στις προτεινόμενες ταινίες.<br>

In [ ]:
vectorizer = TfidfVectorizer(lowercase=True,max_df=100,min_df=10,max_features=10000)
vectorizer.fit(corpus)
corpus_tf_idf_plain = vectorizer.transform(corpus)
print(corpus_tf_idf_plain.shape)
content_recommender(17,10,corpus_tf_idf_plain)

(5000, 7239)
  Movie_ID  Title                                                   Corpus                                             Categories
----------  ------------------------------------------------------  -------------------------------------------------  ----------------------------------------------------------------------------------------
        17  Dogora                                                  As an amorphous alien lifeform annihilated a  ...  "Science Fiction",  "Japanese Movies",  "Creature Film",  "World cinema"
      1065  If Winter Comes                                         Set in the English village Penny Green in 193 ...  "War film"
      4163  Creature                                                The film starts with Richland , who was sent ...   "Thriller",  "Science Fiction",  "Horror"
       900  Dragon Fist Explosion!! If Gokū Can't Do It, Who Will?  The movie begins with a young boy on a dista ...   "Japanese Movies"
      3850  Frankenstein vs. 

Για την ταινία με ID = 25 παρατηρούμε ότι δεν είναι παρόμοια με τις περισσότερες συστάσεις, αφού αυτή αποτελεί μια ρομαντική κωμωδία και οι υπόλοιπες κατα πλειονότητα είναι "Horror" και "Thriller"

In [ ]:
vectorizer = TfidfVectorizer(lowercase=True,max_df=100,min_df=10,max_features=10000)
vectorizer.fit(corpus)
corpus_tf_idf_plain = vectorizer.transform(corpus)
print(corpus_tf_idf_plain.shape)
content_recommender(25,10,corpus_tf_idf_plain)

(5000, 7239)
  Movie_ID  Title                           Corpus                                             Categories
----------  ------------------------------  -------------------------------------------------  ------------------------------------------------------------------------------------------------------------------------------------
        25  (Untitled)                      Set in the artsy Chelsea, this satirical film ...  "Romantic comedy",  "Romance Film",  "Satire",  "Comedy",  "Indie"
      1844  The Nature of the Beast         The story is set in Southern California and  ...   "Thriller",  "Horror",  "Psychological thriller",  "Mystery",  "Drama",  "War film",  "Crime Thriller",  "Action",  "Spy"
       709  33 Scenes from Life             The Polish artist Julia and her husband Piotr ...  "Drama"
      1874  The Bride of the Regiment       As they are emerging from the church followin ...  "Musical",  "Pre-Code"
      2685  Pulse                           When Josh

In [ ]:
corpus_preprocessed_ = [stemSentence(d) for d in corpus_preprocessed]
corpus_preprocessed_ = np.array(corpus_preprocessed_)

In [ ]:
vectorizer = TfidfVectorizer(lowercase=True,max_df=100,min_df=10,max_features=1000)
vectorizer.fit(corpus_preprocessed_)
corpus_tf_idf_plain_ = vectorizer.transform(corpus_preprocessed_)

content_recommender(25,10,corpus_tf_idf_plain_)

  Movie_ID  Title                           Corpus                                             Categories
----------  ------------------------------  -------------------------------------------------  ------------------------------------------------------------------------------------------------------------------------------------
        25  (Untitled)                      Set in the artsy Chelsea, this satirical film ...  "Romantic comedy",  "Romance Film",  "Satire",  "Comedy",  "Indie"
      1844  The Nature of the Beast         The story is set in Southern California and  ...   "Thriller",  "Horror",  "Psychological thriller",  "Mystery",  "Drama",  "War film",  "Crime Thriller",  "Action",  "Spy"
      1874  The Bride of the Regiment       As they are emerging from the church followin ...  "Musical",  "Pre-Code"
       709  33 Scenes from Life             The Polish artist Julia and her husband Piotr ...  "Drama"
      4651  A Midsummer Night's Sex Comedy  It is the early 1900s.

Όταν εφαρμόζουμε stemming πάνω στο corpus, παρατηρούμε μία μικρή βελτίωση στις συστάσεις, όμως ακόμη υπάρχουν ταινίες-στόχοι για τις οποίες οι συστάσεις δεν είναι σχετικές.

Από τα παραπάνω αποτελέσματα μπορούμε να αναγνωρίσουμε τα πλεονεκτήματα και τα μειονεκτήματα ενός recommender βασισμένο στο tfidf. Αρχικά, με τον tfidf είναι πολύ απλή η προεπεξεργασία του corpus, δίνοντας μας πολλές διαφορετικές επιλογές και παραμέτρους, που βοηθάνε σε αυτή τη διαδικασία. Ακόμη, κάνοντας rescale την συχνότητα με την οποία εμφανίζονται οι λέξεις που κυριαρχούν στο corpus (πχ "The") δίνει σημασία στις λέξεις που επηρεάζουν την σημασία της περιγραφής. Ωστόσο αγνοεί την σημασιολογική σχέση των λέξεων, που στη δικιά μας περίπτωση, είναι αυτό που ζητάμε, και σε αυτό βασιζόμαστε για να ελέγξουμε αν οι συστάσεις είναι σωστές. Αυτός είναι και ο λόγος που για ορισμένες ταινίες-στόχους οι συστάσεις είναι λανθασμένες, αφού αγνοείται η σημασία μίας λέξης που χρησιμοποιείται και ελέγχεται μόνο η ύπαρξη της.

### Σύγκριση και σχολιασμός με recommenders βασισμένων στο Word2Vec

In [ ]:
content_recommender(25, 10, w2v)

  Movie_ID  Title                               Corpus                                             Categories
----------  ----------------------------------  -------------------------------------------------  -----------------------------------------------------------------------------------------------------------------------------------
        25  (Untitled)                          Set in the artsy Chelsea, this satirical film ...  "Romantic comedy",  "Romance Film",  "Satire",  "Comedy",  "Indie"
      3873  Jubilee                             In Jubilee, Queen Elizabeth I  is transported ...  "Fantasy",  "Time travel",  "Experimental film",  "Punk rock",  "Indie"
      3070  Topkapi                             The film opens with Elizabeth Lipp  visiting ...   "Crime Fiction",  "Heist",  "Crime Comedy",  "Caper story",  "Action/Adventure",  "Drama",  "Comedy",  "Film adaptation",  "Action"
      1816  Follow That Camel                   His reputation brought into disrepute by Ca

Παρατηρούμε ότι σε όλες τι περιπτώσεις που μπορέσαμε να ελέγξουμε, οι ταινίες που συστήνει το σύστημα μας (βασισμένο στο W2V) είναι πάντα παρόμοιες μεταξύ τους. Παραπάνω φαίνεται η ταινία με ID = 25 στην οποία ο recommender βασισμένος μόνο στο tfidf αποτύγχανε να προτείνει σχετικές ταινίες. <br>